<a style="padding-left:600px" href="https://colab.research.google.com/github/emasri/trd/blob/master/td.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Internet speed

In [ ]:
#@markdown

!sudo apt install speedtest-cli
!speedtest-cli

#### Start drive

In [ ]:
#@markdown
import enum
import os, time,  re, IPython, urllib
#from genericpath import exists

os.environ['TZ'] = 'Turkey'
time.tzset()
if not os.path.exists("/content/drive"):
  from google.colab import drive
  drive.mount('/content/drive')
  downloads_dict = {}
  completed = []
  active=[]
  uncompleted= []
  watched_movies_list = []
  watched_list_url = "https://www.imdb.com/search/title/?title_type=feature&lists=ls047519430&view=simple&sort=alpha,asc&count=250&start="
  page1 = watched_list_url + "0"
  page2= watched_list_url + "251"

  #from os import makedirs makedirs("/root/.config/rclone", exist_ok = True)
  
def auto_reconnect(): 
  from google.colab import output
  display(IPython.display.Javascript('''function ClickConnect(){btn = document.querySelector("colab-connect-button") if (btn != null){console.log("Click colab-connect-button"); btn.click() } btn = document.getElementById('ok') if (btn != null){console.log("Click reconnect"); btn.click() }} setInterval(ClickConnect,60000)'''))

def dump(obj):
  for attr in dir(obj):
    print("                   %s =   %r" % (attr, getattr(obj, attr)))

Save_Base = "/content/drive/Shareddrives/SM1/Downloads"  #@param {type:"string"}
Movies_Path = "/content/drive/Shareddrives/SM1/Library/Movies" #@param {type:"string"}
Shows_Path = "/content/drive/Shareddrives/SM1/Library/TV Shows" #@param {type:"string"}
Part = "1" #@param {type:"string"}

drive_path = "" 
done_path = "/content/done"

Debug = False #@param {type:"boolean"}
if Debug:
  drive_path = "/content/debug"
else:
  drive_path = Save_Base + "/d-0" + Part

temp_save="/content/d-0" + Part


os.makedirs("/content/done", exist_ok = True)
os.makedirs(temp_save, exist_ok = True)
os.makedirs("/content/Added", exist_ok = True)

#@markdown 
Test_drive = False #@param {type:"boolean"}
if Test_drive:
  direct_url = "https://download.visualstudio.microsoft.com/download/pr/c4c86d02-a47b-4bd4-b73d-ec3be19e5245/76c673e22a120464c95f85bef342a361/aspnetcore-runtime-6.0.7-win-x64.exe" 
  output_dir = "/content/test" 
  !wget "$direct_url" -O "$output_dir"  
  !mv "/content/test" "/content/drive/My Drive"

print("Local temporary saving path  " + temp_save)
print("Local done saving path  " + done_path)
print("Drive main saving path:  " + drive_path)
print("Drive movie saving path:  " + Movies_Path)
print("Drive show saving path:  " + Shows_Path)

def filter_dirs(ps):
  for p in ps:
    if p[0] == '.':
      ps.remove(p)
  return ps

def print_files(drs, range_start , range_end):
  for i in range(range_start, range_end):
    print("   {0}-  {1}".format(i+1, drs[i])) 
    
def RoundTo1(n) :
  if n < 1 and (n-0.9765625) >= 0 :
    return 1
  else: 
    return n

def data_str(byte_val):
  size_gb = ByteToGB(byte_val)
  if (size_gb >= 1) :
    return "{:.1f} GB".format(size_gb)
  else:
    size_mb = ByteToMB(byte_val)
    return "{:.1f} MB".format(size_mb)

def speed_str(byte_val):
  size_mb = ByteToMB(byte_val)
  if (size_mb >= 1) :
    return "{:.1f} MB".format(size_mb)
  else:
    return "{:.1f} KB".format(ByteToKB(byte_val))

ByteToGB = lambda n: RoundTo1(n / 1073741824)
ByteToMB = lambda n: RoundTo1(n/ 1048576)
ByteToKB = lambda n: RoundTo1(n/ 1024)
contains = lambda s1, s2: s2 in s1.lower()

def movie_name_trans(name): 
  name = name.strip().replace("&","and")
  dgts = re.findall(rx, name)
  if '1080' in dgts:
    dgts.remove('1080')
  if dgts:  
    if len(dgts) == 1:
      name = name.split(dgts[0])[0]
    else:
      name = name.split(dgts[1])[0] 
  table = name.maketrans("",""," '(#$%-,.;:!?)")
  trans = name.translate(table).lower()
  return trans

def show_name_trans(name): 
  name = name.strip()
  dgts = re.findall(rx, name)
  if dgts:
    name = name.split(dgts[0])[0].lower()
  else :
    name = name.lower()
  table = name.maketrans("",""," '(#$%&-,.;:!?)")
  trans = name.translate(table)
  return trans

def movie_exists(name):
  movies_dir = filter_dirs(next(os.walk(Movies_Path))[1])
  movies = [movie_name_trans(x) for x in movies_dir]
  name_s = movie_name_trans(name)
  if name_s in movies :
    return True
  else:
    return False


def show_exists(name):
  name_s = name_show(name)
  path = Shows_Path + "/" + name_s
  return os.path.exists(path)

# name_s = name_movie(name)
# path = Movies_Path + "/" + name_s
# return os.path.exists(path)

# shows_dir = filter_dirs(next(os.walk(Shows_Path))[1])
# shows = [show_name_trans(x) for x in shows_dir]
# name_s = show_name_trans(name)
# if name_s in shows :
#   return True
# else:
#   return False

def episode_exists(name):
  show= name_show(name)
  season= name_season(name)
  episode= name_episode(name)
  p0 = Shows_Path + "/" + show + "/" + season  + "/" + episode
  return os.path.exists(p0)

def season_exists(name):
  show= name_show(name)
  season= name_season(name)
  p0 = Shows_Path + "/" + show + "/" + season
  return os.path.exists(p0)

def get_magnet_name(magnet):
  return urllib.parse.unquote(magnet.split("dn=")[1].split("&tr=")[0], encoding='utf-8', errors='replace').replace('+',".")

skip_list = ['.txt',
              '.inf',
              '.info',
              'nfo',
              'www.yts',
              'rarbg_do_not']



def add_torrent_file(ti, type):
  #downloads = ses.get_torrents()
  name =ti.name()
  print("Downloading: " + name)
  params = {
  "save_path": temp_save,
  "ti": ti}
  print("Files: {0}".format(ti.num_files()))
  print("Metadata size: {0}".format(speed_str(ti.metadata_size())))
  print("Download files:\n")
  files = ti.files()
  handle = ses.add_torrent(params)
  key = str(handle.info_hash())

  if key in downloads_dict:
    print(" Torrent already added.\n")
    return

  download = {
    "type": type,
    "handle": handle
  } 
  downloads_dict[key] = download
  handle.auto_managed(True)
  active.append(handle)

  for index in range(files.num_files()):
    if any(contains(files.file_name(index), check) for check in skip_list):
      handle.file_priority(index,0)
    else: 
      handle.file_priority(index,6)
      fs_str= data_str(files.file_size(index))
      fi_str = "   {:3d}-  ".format(index) + fs_str
      print(fi_str, files.file_name(index))
  print()
  
def add_torrent_magnet(magnet_link, name, type):
  params = {"save_path": temp_save}
  handle = lt.add_magnet_uri(ses, magnet_link, params)    
  if not handle.is_valid():
    print(" Invalid torrent  {0}".format(name))
    return 

  key =  str(handle.info_hash())
  if key in downloads_dict:
    print(" Torrent already added.\n")
    return
  download = {
    "type": type,
    "handle": handle
  }
  handle.auto_managed(True)
  downloads_dict[key] = download
  active.append(handle)
  
  if handle.has_metadata():
    ti = handle.get_torrent_info()
    files = ti.files()
    print()
    for index in range(files.num_files()):
      if any(contains(files.file_name(index), check) for check in skip_list):
        handle.file_priority(index,0)
      else: 
        handle.file_priority(index,6)
        fs_str= data_str(files.file_size(index))
        fi_str = "   {:3d}-  ".format(index) + fs_str
        print(fi_str, files.file_name(index))
  print()
  return handle

def replace_download(path):
  if os.path.exists(path):
    parent = os.path.dirname(path)
    rp = parent + "/" + ".replace"
    os.makedirs(rp, exist_ok=True)
    !mv "$path" "$rp"

def add_local_files(type, replace= False):
  print("Loading torrent files in /content directory:\n")
  fls = next(os.walk("."))[2]
  for fl in fls:
    splited = os.path.splitext(fl)
    if splited[1] == ".torrent":
      ti = lt.torrent_info(fl)
      name =ti.name()
      if not ti.is_valid():
        print("Invalid torrent.")
        continue
      if replace:
        if type == Types.Movie: 
          name = name_movie(name)
          path = Movies_Path + "/" + name
        elif type == Types.Season:
          name = name_season(name)
          showname = name_show(name)
          path = f"{Shows_Path}/{showname}/{name}"
        elif type == Types.Episode:
          name = name_episode(name)
          showname = name_show(name)
          season = name_season(name)
          path = f"{Shows_Path}/{showname}/{season}/{name}"
        replace_download(path)
      else:
        if type == Types.Movie and movie_exists(name):
          print(name + " is already downloaded.")
          continue
        elif type == Types.Season and season_exists(name):
          print(name + " is already downloaded.")
          continue
        elif type == Types.Episode and episode_exists(name):
          print(name + " is already downloaded.")
          continue
      add_torrent_file(ti, type)
      !mv "$fl" "/content/Added"

def filterfiles():
  for h in active:
    if(h.has_metadata()):
      torinfo = h.get_torrent_info()
      files = torinfo.files()
      for x in range(files.num_files()):
        if any(contains(files.file_name(x), check) for check in skip_list):
          h.file_priority(x,0)
        else: 
          h.file_priority(x,5)
  total_download = 0
  print("Download files:")
  for h in active:
    print("")
    print("  "+h.name())
    if(h.has_metadata()):
      priorities = h.file_priorities()
      torinfo = h.get_torrent_info()
      files = torinfo.files()
      status= h.status()
      file_sizes = status.total_wanted
      total_download += file_sizes
      size_str = data_str(file_sizes)
      print("  Total download size: {}\n".format(size_str))
      print("  Index Size    Name")
      fi_str=""
      for x in range(files.num_files()):
        if (priorities[x]>0):
          fs_str= data_str(files.file_size(x))
          fi_str = "   {:3d}-  ".format(x) + fs_str
          print(fi_str,files.file_name(x))
    else:
      print("  Metadata not recieved yet.\n")
  all_total_str = data_str(total_download)
  print("Total download: ", all_total_str)

def is_empty(path):  
  if os.path.exists(path):
    dircontent = os.listdir(path)
    if len(dircontent) == 0 or (len(dircontent) == 1 and dircontent[0] == '.ipynb_checkpoints' ):
        return True
    else:
        return False
  else:
    return False

def shave (handle, path, maindir) :
  ti = handle.get_torrent_info()
  files = ti.files()
  priorities = handle.file_priorities();
  
  for i in range(len(priorities)):
    file_path = path + "/" + files.file_path(i)
    if os.path.isfile(file_path) and priorities[i] == 0:
      !rm "$file_path"
      dirpath = os.path.dirname(file_path)
      if is_empty(dirpath):
        !rm -r "$dirpath"

  dircontent = next(os.walk(maindir))[1]
  for path in dircontent:
    if is_empty(path):
      !rm -r "$dirpath"


def after_shave (path, newname) :
  dircontent = next(os.walk(path))[2]
  for file in dircontent:
    split = os.path.splitext(file)
    ext = split[1]
    p1 = path + "/" + file
    if  any(contains(file, check) for check in skip_list):
      !rm "$p1"

    elif  (ext == ".mp4" or ext == ".mkv") and not split[0] == newname:
      p2 = path + "/" + newname + ext
      !mv "$p1" "$p2"

def clean_movie(path, name):
  print(name)
  newname = name
  if "." in name or "1080" in name:
    newname = name_movie(name)
    p1 = path
    p2 = Movies_Path + "/" + newname
    !mv "$p1" "$p2"
    path = p2
    
  after_shave(path, newname)

def clean_movies_dir() :
    dir_content = next(os.walk(Movies_Path))[1]
    for dir in dir_content :
      path = Movies_Path + "/" + dir
      clean_movie(path, dir)

def clean_tv (path):
  dircontent = next(os.walk(path))[1]
  for season in dircontent:
    check = season.lower()
    if len(season) > 3 and ("season" in check or "s0" in check):
      newname = name_season(season)
      if newname != season:
        p0 =  path + "/" + season
        p1 = path + "/" + newname
        !mv "$p0" "$p1"
        season = newname
    seasonpath = path + "/" + season
    clean_season(seasonpath)

def clean_season(path):
  if not os.path.exists(path):
    return
  files = next(os.walk(path))[2]
  for file in files:
    splitted = os.path.splitext(file)
    eps = splitted[0]
    ext = splitted[1]
    p1 = path + "/" + file
    if ext == ".mkv" or ext == ".mp4" or ext == ".srt":
      newname = name_episode(eps) + ext
      if newname != file:
        p2 = path + "/" + newname
        !mv "$p1" "$p2" 
    elif any(contains(file.lower(), check) for check in skip_list):
      !rm "$p1" 
      
rx = r"\d{4}"
def name_show(name): 
  name = name.strip()
  table = name.maketrans("."," ","'()")
  trans = name.translate(table)
  dgts = re.findall(rx, trans)
  if '1080' in dgts:
    dgts.remove('1080')
  if dgts:  
    return trans.split(dgts[0])[0] + dgts[0]
  elif re.search(rx2, trans) != None:
    return re.split(rx2, trans)[0].strip()
  else:
    return trans

def name_movie(name): 
  name = name.strip()
  table = name.maketrans("."," ","()")
  trans = name.translate(table)
  digits = re.findall(rx, trans)
  if '1080' in digits:
    digits.remove('1080')
  if len(digits) >= 2:
       return trans.split(digits[1])[0] + digits[1]   
  elif digits:  
      return trans.split(digits[0])[0] + digits[0]
  else:
    return trans

rx1 = r"season\d{1,2}"
rx2 = r"S\d{1,2}|s\d{1,2}"
def name_season(name):
  trans = name.replace(" ","").lower()
  checkname = re.search(rx1, trans)
  if checkname != None:
    return seasons[checkname.group()].title()
  else:
    return re.search(rx2, trans).group().title()

def name_episode(name):
  if "1080" in name:
    name = name.split("1080")[0]
  elif "720p" in name:
    name = name.split("720p")[0]
  table = name.maketrans("", "", "()")
  name = name.translate(table)
  if name[-1] == ".":
    name = name[:-1]
  return name.strip().title()

def clean_shows():
  shows = next(os.walk(Shows_Path))[1]
  for show in shows:
    print(show)
    path = Shows_Path + "/" + show
    clean_tv(path)

seasons = {"season1":"S01", "season2":"S02", "season3":"S03", "season4":"S04", "season5":"S05", "season6":"S06", "season7":"S07", "season8":"S08", "season9":"S09", "season10":"S10", "season11":"S11", "season12":"S12", "season13":"S13", "season14":"S14"}

class Types (enum.Enum):
  General = 1 
  Movie = 2
  Show = 3
  Season = 4
  Episode = 5

class result:
  def __init__(this, title, size = 0, date = "", seeders = "", leechers = "", uploader = "", href = "", state = "" ):
    this.title = title
    this.size = size
    this.date = date 
    this.seeders = seeders
    this.leechers = leechers
    this.uploader = uploader
    this.href = href
    this.state = state

sort_seeders = "seeders/desc"
sort_size_asc = "size/asc"
sort_size_desc = "size/desc"
sort_time = "time/desc"

def DownloadRarbg(search, categories, Type):
  print(f"Searching for:  {search}  in rarbg.com.\n")
  results = rarbg.search(search_string=search, categories=categories)
  if not results:
    print("No results were found.\n")
    if not category_movie_h264 in categories:
      print("Trying to search in rarbg x264 category..\n")
      categories.append(category_movie_h264)
      results = rarbg.search(search_string=search, categories=categories)
    else:
      return
  if results:
    if len(results) == 1:
      print("Adding torrent: {}".format(results[0].filename))
      add_torrent_magnet(results[0].download, search, Type)
      print(" ............\n")
      return
    else: 
      print("     {:<70}{:<10}{:<10}".format("Title", "", ""))
      for i in range(len(results)):
        print(" {}- {:<70} ".format(i + 1 , results[i].filename))

      print("\nPlease choose one of the results to download:             Format [single]     Enter s to skip\n")
      choice = input("")
      if choice == "s" or choice == "q":
        return
      choice = int(choice) - 1
      if choice < 0 or choice >= len(results):
        print("Choice out of range.\n")
        return
      print(f"Adding torrent: {results[choice].filename}")
      add_torrent_magnet(results[choice].download, search, Type)
      print(" ............\n")
    return
  else:
    print("No results were found.\n")
    return

def check_download(type0, name):
  if type0 == Types.Movie:
    name = name_movie(name)
    return movie_exists(name)
  if type0 == Types.Show:
    name = name_show(name)
    return show_exists(name)

def search(Site, Search, sort, Type, replace = False, Page = 1, categories = []):
  if Site == "1337x":
    page = max(Page, 1)
    while page != 0 :
      page = scrap1337x(Search, sort, Type, replace, page)
  else:
    DownloadRarbg(Search, categories, Type)

def scrap1337x(search, sort, Type, replace = False, page = 1):
  search = search.replace(" ","+").replace("'", " ")
  url = f"https://www.1337xx.to/sort-search/{search}/{sort}/{page}"
  print("Page: {} \n".format(url))
  response = requests.get(url, headers= {"Accept-Language": "en-US"})
  if response.status_code == 200 :  
    soup = BeautifulSoup(response.text, 'html.parser' )
    main_element = soup.find("tbody")
    if main_element is None:
      print("null search")
      return
    all_res = main_element.findAll("tr")
    title = "" 
    size = ""
    uploader = ""
    seeders = ""
    leechers = ""
    results = [] 
    if all_res:
      print("    {:<120}{:^12}    {:^15}{:^21}{:^4}{:>14}\n".format("Title", "Size", "Date", "Uploader", "Seeders", "Leechers"))

    for i in  range(len(all_res)):
      alldata= all_res[i].findAll("td")
      title = alldata[0].findAll("a")[1].text
      size = alldata[4].text
      date = alldata[3].text
      seeders = alldata[1].text
      leechers = alldata[2].text
      uploader = alldata[5].a.text
      if check_download(Type, title):
        state = "Downloaded"
      else:
        state = ""
      print("{:<2}- {:<118}{:>11}       {:<16}{:>12}{:>12}{:>13}{:>14}".format(i + 1, title.strip(), size, date, uploader, seeders, leechers, state))
      href = "https://www.1337xx.to" + alldata[0].findAll("a")[1]['href']
      res = result(title= title, size= size, date= date, seeders= seeders, leechers= leechers, uploader= uploader, href= href, state=state)
      results.append(res)

    print("\nPlease choose a result to download:\n        Enter s or q to quit, n for the next page, p for the previous page.\n")
    choice = input("")
    if choice == 'n':
      return page + 1
    if choice == 'p':
      if page > 1:
        return page - 1
      else:
        return page
    if choice == "s" or choice == "q":
      print("\nExiting..\n")
      return 0
    if ',' in choice:
      choices = choice.split(',')
    else:
      choices = [choice]
    for choice in choices:
      select = results[int(choice) - 1]
      print(f"\n Downloading file for and adding {select.title} :\n")
      url = select.href
      download = requests.get(url)
      soup = BeautifulSoup(download.text, "html.parser")
      hash = soup.find(class_="infohash-box").span.text.strip()                                
      download_link = f"https://itorrents.org/torrent/{hash}.torrent"
      file = "/content/{}.torrent".format(select.title)
      !wget "$download_link" -O "$file" &> /dev/null
      add_local_files(Type, replace)
    print("Done.\n")
    return 0
  else:
    print(f" An error occured while trying to access 1337x.com  status: {response.status_code}")
    return 0

import unicodedata

def normalize(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

def search_title(title):
  table = title.maketrans("/"," ",":'(,?)")
  title = title.translate(table)
  return normalize(title.replace(" I ", " ").replace("&", "and"))

def scrap_watched_list(text):
  soup = BeautifulSoup(text,'html.parser')    
  scraped = soup.find("div",{"class":"lister-list"})
  if not scraped :
      return
  all_items = scraped.findAll(class_="lister-item-header")
  for i in range(len(all_items)):
      year = all_items[i].findChild(class_="lister-item-year").text
      title = "{0} {1}".format(all_items[i].a.text, year)
      title = search_title(title)
      watched_movies_list.append(title)
  return

class imdb_result:
  def __init__(self, title, runtime, genres, rating, votes, director, staring):
    self.title = title
    self.runtime = runtime
    self.rating = rating
    self.votes = votes
    self.director = director
    self.staring = staring
    self.genres = genres
    
auto_reconnect()

#### Start Session

In [ ]:
#@markdown
 
!python -m pip install --upgrade pip setuptools wheel &> /dev/null
!python -m pip install lbry-libtorrent &> /dev/null
!apt install python3-libtorrent &> /dev/null

!pip install rarbgapi &> /dev/null
!pip3 install requests &> /dev/null
!pip3 install beautifulsoup4 &> /dev/null

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)

import requests
from bs4 import BeautifulSoup

reponse1 = requests.get(page1, headers = {"Accept-Language": "en-US"})
scrap_watched_list(reponse1.text)
reponse2 = requests.get(page2, headers = {"Accept-Language": "en-US"})
scrap_watched_list(reponse2.text)

import rarbgapi

category_tv = rarbgapi.RarbgAPI.CATEGORY_TV_EPISODES  
category_tv_hd = rarbgapi.RarbgAPI.CATEGORY_TV_EPISODES_HD  
category_movie_h264 = rarbgapi.RarbgAPI.CATEGORY_MOVIE_H264_1080P  
category_movie_h265 = rarbgapi.RarbgAPI.CATEGORY_MOVIE_H265_1080P


#### Settings

In [ ]:
#@markdown

Announce_To_All_Tiers = True #@param {type:"boolean"}
Announce_To_All_Trackers = True #@param {type:"boolean"}
Active_Limit = 5 #@param {type:"integer"}
Active_Downloads = 3 #@param {type:"integer"}
Active_Seeds = 1 #@param {type:"integer"}
Active_Checking = 3 #@param {type:"integer"}
Connections_Limit = 2330 #@param {type:"integer"}
Connection_Speed = 80 #@param {type:"integer"}
Connections_Slack = 50 #@param {type:"integer"}
File_Pool_Size = 20 #@param {type:"integer"}
Half_Open_Limit = 100 #@param {type:"integer"}
Max_Rejects = 8 #@param {type:"integer"}
Max_Peerlist_Size = 5000 #@param {type:"integer"}
Max_Paused_Peerlist_Size = 2000 #@param {type:"integer"}
Max_Failcount = 3 #@param {type:"integer"}
Min_Announce_Interval = 30 #@param {type:"integer"}
Min_Reconnect_Time = 20 #@param {type:"integer"}
Request_Timeout = 10 #@param {type:"integer"}
Smooth_Connects = False #@param {type:"boolean"}
Send_Buffer_Watermark_MB = 5  #@param {type:"integer"}
Seed_Time_Limit = 60 #@param {type:"integer"}
Tracker_Completion_Timeout = 30 #@param {type:"integer"}
Upload_Rate_Limit_KB = 0 #@param {type:"integer"}
User_Agent = '"Utorrent"/3.5.5' #@param {type:"string"}

sett = {
    'allow_multiple_connections_per_ip': True,
    'active_downloads': Active_Downloads,
    'active_seeds':Active_Seeds,
    'active_checking': Active_Checking,
    'active_limit':Active_Limit,
    'active_seeds':Active_Seeds,    
    'announce_to_all_tiers': Announce_To_All_Tiers,
    'announce_to_all_trackers':Announce_To_All_Trackers,
    'allow_multiple_connections_per_ip':True,
    'auto_sequential':True,
    'auto_manage_interval': 30,
    'connections_limit':Connections_Limit,
    'connection_speed': Connection_Speed,
    'connections_slack':Connections_Slack,
    'close_redundant_connections':True,
    'file_pool_size':File_Pool_Size,
    'local_service_announce_interval':200,
    'max_rejects':Max_Rejects,
    'max_peerlist_size': Max_Peerlist_Size,
    'max_paused_peerlist_size': Max_Paused_Peerlist_Size,
    'max_failcount':Max_Failcount,
    'min_announce_interval':Min_Announce_Interval,
    'min_reconnect_time': Min_Reconnect_Time,
    'peer_timeout':20,
    'request_timeout':Request_Timeout,
    'half_open_limit': Half_Open_Limit,
    'send_buffer_watermark':Send_Buffer_Watermark_MB*1024*1024,
    'smooth_connects':Smooth_Connects,
    'strict_end_game_mode':True,
    'seed_time_limit': Seed_Time_Limit,
    'tracker_completion_timeout':Tracker_Completion_Timeout,
    'upload_rate_limit':Upload_Rate_Limit_KB*1024,
    'user_agent': User_Agent,
    #'inactivity_timeout':20,
    #'ban_web_seeds':False,
    }

ses.apply_settings(sett)

#'inactivity_timeout':20,
#'ban_web_seeds':False,
  

#### Load uncompleted

In [ ]:
#@markdown 

Load_From = "/content/drive/Shareddrives/SM1/Downloads/.uncomplete" #@param {type: "string"}

def load_uncompleted(path):
  content = next(os.walk(path))[1]
  if content:
    print(" Choose a directory to load:")
    for i,c in enumerate(content[:]):
      print("   {0}-  {1}".format(i+1, c))
    print()
    choice = input("")
    if "q" == choice.lower() or "s" == choice.lower():
      return
    elif "," in choice:
      choices = choice.split(",")
    else:
      choices = [choice]
      
    for select in choices:
      select = int(select) - 1
      if select >= len(content) or select < 0 :
        print("\nThe choice is out of range.\n")
      else:
        print("Loading {0} into {1}\n".format(content[select], temp_save))
        p0 = path + "/" + content[select]
        !cp -r "$p0" "$temp_save"
        uncompleted.append(content[select])
        print("\n   ...................\n          Done\n")

if Load_From:
  load_uncompleted(Load_From)


#### Review settings

In [ ]:
#@markdown

settings = ses.get_settings()
setn =[]

for x in settings:
  setn.append(str(x))

setn.sort()

for x in setn:
  print("'{0}': {1},".format(x,settings[x]))

#### Search

In [ ]:
import requests
from bs4 import BeautifulSoup 
import rarbgapi
rarbg = rarbgapi.RarbgAPI()
Type = Types.Episode #@param ["Types.General", "Types.Movie", "Types.Show", "Types.Season", "Types.Episode"] {type:"raw"}
Site = "1337x" #@param ["1337x", "rarbg"] {type:"string"}
Replace = False #@param {type:"boolean"}

#@markdown 

Search = "" #@param {type:"string"}
Page =  1 #@param {type:"integer"}

#@markdown 

Movie265_Category = True #@param {type:"boolean"}
Movie264_Category = False #@param {type:"boolean"}
TV_Category = False #@param {type:"boolean"}
TV_HD_Category = False #@param {type:"boolean"}

#@markdown 

By_Seeders = False #@param {type:"boolean"}
By_Size_Descending = False #@param {type:"boolean"}
By_Size_Ascending = "" #@param {type:"boolean"}
By_Time = "" #@param {type:"boolean"}


sort = sort_seeders
if By_Size_Descending:
  sort = sort_size_desc
elif By_Size_Ascending:
  sort = sort_size_asc
elif By_Time:
  sort = sort_time

categories = []
if Movie265_Category :
  categories.append(category_movie_h265)
if Movie264_Category :
  categories.append(category_movie_h264)
if TV_Category :
  categories.append(category_tv)
if TV_HD_Category :
  categories.append(category_tv_hd)  

if Search:
  search(Site, Search, sort, Type, Replace, Page, categories)
elif uncompleted:
  for item in uncompleted:
    print(f"Downloading: {item}\nSelect a source:\n 1 - 1337x\n 2 - rarbg\n")
    select = input("")
    if select.lower() == "s":
      continue
    elif select.lower == "q":
      break
    elif select == "1":
      search("1337x", item, sort, Type, Replace, Page= Page)
    elif select == "2":
      search("rarbg", item, sort, Type, categories= categories )    

#### Auto download

In [ ]:
import requests
from bs4 import BeautifulSoup 
import rarbgapi
rarbg = rarbgapi.RarbgAPI()
#@markdown

Type = Types.Movie #@param ["Types.General", "Types.Movie", "Types.Show", "Types.Season", "Types.Episode"] {type:"raw"}

Years="1990-2000" #@param {type:"string"}
Ratings="7.2-" #@param {type:"string"}
Min_Votes="70000" #@param {type:"string"}
Max_Runtime_M="190" #@param {type:"string"}
IMDB_Lists="" #@param {type:"string"}
Top_250=False #@param {type:"boolean"}
Top_1000=False #@param {type:"boolean"}
Oscar_Winner=False #@param {type:"boolean"}
Best_Picture_Oscar=False #@param {type:"boolean"}
Best_Director_Oscar=False #@param {type:"boolean"}
English_Only=True #@param {type:"boolean"}
Animations=False #@param {type:"boolean"}
International_Movies=False #@param {type:"boolean"}
Indian_Movies=False #@param {type:"boolean"}
Sort_Date=True #@param {type:"boolean"}
Sort_Rating=False #@param {type:"boolean"}
Sort_Alphabatically=False #@param {type:"boolean"}
Sort_Ascending=False #@param {type:"boolean"}
Sort_Descending=True #@param {type:"boolean"}
Filter_Downloaded = True #@param {type:"boolean"}
Filter_Watched = False #@param {type:"boolean"}

#@markdown

l337x_Download = True #@param {type:"boolean"}
Rarbg_Download = True #@param {type:"boolean"}
#@markdown

Movie265_Category = True #@param {type:"boolean"}
Movie264_Category = False #@param {type:"boolean"}
TV_Category = False #@param {type:"boolean"}
TV_HD_Category = False #@param {type:"boolean"}

category_tv = rarbgapi.RarbgAPI.CATEGORY_TV_EPISODES  
category_tv_hd = rarbgapi.RarbgAPI.CATEGORY_TV_EPISODES_HD  
category_movie_h264 = rarbgapi.RarbgAPI.CATEGORY_MOVIE_H264_1080P  
category_movie_h265 = rarbgapi.RarbgAPI.CATEGORY_MOVIE_H265_1080P

exclude="%21"
international = "ls560608427"
indian= "ls560456074"
animation = "ls560333050"
watched_movies ="ls047519430"
watched_series = "ls047701980"
imdb = "https://www.imdb.com/title/"
date_sort = "release_date"
rating_sort = "user_rating"
alpha_sort = "alpha"

top250="top_250"
top1000= "top_1000"
oscar_winner="oscar_winner"
best_picture_oscar="best_picture_winner"
best_director_oscar="best_director_winner"

lists=""
imdb_labels = ""
start_year=""
end_year=""
ratings = Ratings.replace("-",",")
sort=""
if not '-' in Years:
  start_year = Years
  end_year= Years
else:
  vals = Years.split("-")
  start_year = vals[0]
  end_year= vals[1]

if Top_250:
  imdb_labels= imdb_labels + top250 + ","

if Top_1000:
  imdb_labels= imdb_labels + top1000 + ","

if Oscar_Winner:
  imdb_labels= imdb_labels + oscar_winner + ","

if Best_Picture_Oscar:
  imdb_labels= imdb_labels + best_picture_oscar + ","

if Best_Director_Oscar:
  imdb_labels= imdb_labels + best_director_oscar 

if "," in IMDB_Lists:
  for item in IMDB_Lists.split(","):
    lists = lists + item + ","

elif IMDB_Lists:
  lists = lists + IMDB_Lists + ","

if English_Only:
    lists = lists + exclude + international + "," + exclude + indian + "," + exclude + animation + "&languages=en"
else:
  if International_Movies:
    lists = international + ","

  if Indian_Movies:
    lists = lists + indian 

  if Animations:
    lists = lists + animation 

if Sort_Date:
  sort = date_sort

elif Sort_Rating:
  sort = rating_sort

elif Sort_Alphabatically:
  sort = alpha_sort

else:
  sort = date_sort

if Sort_Ascending and not Sort_Descending:
  sort = sort + ",asc"

else:
  sort = sort + ",desc"

categories = []
if Movie265_Category :
  categories.append(category_movie_h265)
if Movie264_Category :
  categories.append(category_movie_h264)
if TV_Category :
  categories.append(category_tv)
if TV_HD_Category :
  categories.append(category_tv_hd)  

def AdvancedMovieSearch (s_year, e_year, _ratings, min_votes, max_run, _lists, _sort):
  url=f'https://www.imdb.com/search/title/?title_type=feature&release_date={s_year}-01-01,{e_year}-12-31&user_rating={_ratings}&num_votes={min_votes},&runtime=,{max_run}&my_ratings=exclude&lists={_lists}&groups={imdb_labels}&sort={_sort}&view=advanced&count=250'
  print()
  resp=requests.get(url, headers = {"Accept-Language": "en-US"})
  if resp.status_code==200:
      print("--> Search done successfully.")
      print()
      soup=BeautifulSoup(resp.text,'html.parser')    
      l=soup.find("div",{"class":"lister-list"})
      if not l :
        return
      movie_list = []
      downloaded= []
      all_items = l.findAll(class_="lister-item")
      shift = 0
      for i in range(len(all_items)):
        year = all_items[i].find(class_="lister-item-year").text
        title = "{0} {1}".format(all_items[i].find(class_="lister-item-header").a.text, year)
        title = search_title(title)
        runtime = all_items[i].find(class_="runtime").text
        genres = all_items[i].find(class_="genre").text.strip()
        rating = all_items[i].find(class_="ratings-imdb-rating")["data-value"]
        votes = all_items[i].find("span", {"name":"nv"}).text
        res= imdb_result(title, runtime, genres, rating, votes, "", "")
        if title in watched_movies_list and Filter_Watched:
          continue
        if movie_exists(title):
          downloaded.append(res)
          if Filter_Downloaded:
            continue         
        movie_list.append(res)
      print()
      watch_state=""
      if Filter_Downloaded:
        for item in downloaded:
          if item.title in watched_movies_list:
            watch_state = "WATCHED"
          else:
            watch_state = ""
          print("        {:<70}{:<30}DOWNLOADED{:>10}{:^12}{:15}{:>10}".format(item.title, item.genres, item.runtime, item.rating, item.votes, watch_state))
        #print()
        for index in range(len(movie_list)):
          if movie_list[index].title in watched_movies_list:
            watch_state = "WATCHED"
          else:
            watch_state = ""
          print("  {:>3} - {:<70}{:<29}{:>21}{:^12}{:15}{:>10}".format(index + 1 , movie_list[index].title, movie_list[index].genres, movie_list[index].runtime, movie_list[index].rating, movie_list[index].votes, watch_state))
      else:
        watch_state = ""
        download_state=""
        for index in range(len(movie_list)):          
          if movie_list[index].title in watched_movies_list:
            watch_state = "WATCHED"
          else:
            watch_state = ""          
          if movie_list[index] in downloaded:
            download_state = "DOWNLOADED"
          else:
            download_state = ""
          print("  {:>3} - {:<70}{:<29}{:>15}{:>15}{:^12}{:15}{:>10}".format(index + 1,  movie_list[index].title, movie_list[index].genres,  download_state, movie_list[index].runtime, movie_list[index].rating, movie_list[index].votes, watch_state))
      
      if Rarbg_Download or l337x_Download:
        print("\n--> Choose movie range to download:       Format: [start-end]      [title1,title5..]     [single]      Enter [q] to exit\n")
        start= -1
        end = -1
        rng = input("").lower()
        print()
        if rng == 'q' or rng == "s":
          return
            
        elif "," in rng:
          rngs = rng.split(',')
          for item in rngs:
            if "-" in item:
              in_rngs = item.split('-')
              start = int(in_rngs[0])-1
              end = int((in_rngs[1]))
              if l337x_Download and Rarbg_Download:
                for i in range(start, end):
                  print("Downloading: {}\nSelect source:\n\n   1- 1337x\n   2- rarbg\n".format(movie_list[i].title))
                  select= input("")
                  if select == "s" or select == "q":
                    continue
                  select= int(select)
                  if select == 1:
                    search("1337x", movie_list[i].title, sort_seeders, Type)
                  else:
                    search("rarbg", movie_list[i].title, sort_seeders, Type, categories = categories)
              elif l337x_Download:
                for i in range(start, end):
                  search("1337x", movie_list[i].title, sort_seeders, Type)
              elif Rarbg_Download:    
                for i in range(start, end):
                  search("rarbg", movie_list[i].title, sort_seeders, Type, categories = categories)
            else:
              start = int(item)-1
              if l337x_Download and Rarbg_Download:
                print("Downloading: {}\nSelect source:\n\n   1- 1337x\n   2- rarbg\n".format(movie_list[start].title))
                select= input("")
                if select == "s" or select == "q":
                  continue
                select= int(select)
                if select == 1:
                  search("1337x", movie_list[start].title, sort_seeders, Type)
                else:
                  search("rarbg", movie_list[start].title, sort_seeders, Type, categories = categories)
              elif l337x_Download:
                search("1337x", movie_list[start].title, sort_seeders, Type)
              elif Rarbg_Download:  
                search("rarbg", movie_list[start].title, sort_seeders, Type, categories = categories)  

        elif "-" in rng:
          rngs = rng.split('-')
          start = int(rngs[0])-1
          end = int((rngs[1]))
          if l337x_Download and Rarbg_Download:
            for i in range(start, end):
              print("Downloading: {}\nSelect source:\n\n   1- 1337x\n   2- rarbg\n".format(movie_list[i].title))
              select= input("")
              if select == "s" or select == "q":
                continue
              select= int(select)
              if select == 1:
                  search("1337x", movie_list[i].title, sort_seeders, Type)
              else:
                  search("rarbg", movie_list[i].title, sort_seeders, Type, categories = categories)
          elif l337x_Download:
            for i in range(start, end):
              search("1337x", movie_list[i].title, sort_seeders, Type)
          elif Rarbg_Download:  
            for i in range(start, end):
              search("rarbg", movie_list[i].title, sort_seeders, Type, categories = categories)

        else:
          start = int(rng) - 1
          if l337x_Download and Rarbg_Download:
            print("Downloading: {}\nSelect source:\n\n   1- 1337x\n   2- rarbg\n".format(movie_list[start].title))
            select= input("")
            if select == "s" or select == "q":
              return
            select= int(select)
            if select == 1:
              search("1337x", movie_list[start].title, sort_seeders, Type)
            else:
              search("rarbg", movie_list[start].title, sort_seeders, Type, categories = categories)
          elif l337x_Download:
            search("1337x", movie_list[start].title, sort_seeders, Type)
          elif Rarbg_Download:  
            search("rarbg", movie_list[start].title, sort_seeders, Type, categories = categories)
              
        ses.start_dht()
        print("\n         Done")
  else:
      print("Error accessing the web page {0}".format(resp.state_code))

AdvancedMovieSearch(start_year, end_year,ratings,Min_Votes, Max_Runtime_M, lists, sort)



#### Add torrents

In [ ]:
from google.colab import files
#@markdown

Source = "Local" #@param ["Local", "Upload File", "Magnet link"] {type:"string"}
Type = Types.General #@param ["Types.General", "Types.Movie", "Types.Show", "Types.Season", "Types.Episode"] {type:"raw"}
#@markdown

if Source == "Local" or Source == "Upload File":
  nms = []
  if Source == "Upload File":
    source = files.upload()
    nms = list(source.keys())
    
add_local_files(Type)

if Source == "Magnet link":
  magnet_link = ""
  while True:
    magnet = input("Enter magnet link or type q:  ")
    if magnet_link.lower() == "q":
      break
    name = get_magnet_name(magnet_link)
    if type == Types.Movie and movie_exists(name):
      print(name + " is already downloaded.")
      continue

    if type == Types.Episode and episode_exists(name):
      print(name + " is already downloaded.")
      continue

    if type == Types.Season and season_exists(name):
      print(name + " is already downloaded.")
      continue
    add_torrent_magnet(magnet, name, Type)   

ses.start_dht()
print('')
print("Save path: "+ drive_path)
print("Temp Save: " + temp_save)

#### Download metadata

In [ ]:

#@markdown 

#@markdown 

print("Downloading metadata")
checks=[]
for h in active:
  h.resume()
  checks.append(h)

while (checks):
  i = input("enter anything to exit  ")
  if i :
    break
    
  for h in checks:
    if h.has_metadata() :
      h.pause()
      checks.remove(h)
      print("Enter anything to quit.")
  time.sleep(1)

for h in active:
  if(h.has_metadata()):
    tor_info = h.get_torrent_info()
    files = tor_info.files()
    for x in range(files.num_files()):
      h.file_priority(x,5)

time.sleep(2)

#@markdown

total_download = 0

print("Download files:")
for h in active:

  print("")
  print("  "+h.name())

  if(h.has_metadata()):

    priorities = h.file_priorities()
    tor_info = h.get_torrent_info()
    files = tor_info.files()
    file_sizes = tor_info.total_size()
    total_download += file_sizes
    size_str = data_str(file_sizes)

    print("  Total download size: " + size_str)
    print("  Index Size    Name")

    fi_str=""
    for x in range(files.num_files()):
      fs_str= data_str(files.file_size(x))
      fi_str = "   {:3d}-  ".format(x) + fs_str
      print(fi_str,files.file_name(x))
  else:
    print("  Metadata not recieved yet.")
    
print("")
all_total_str = data_str(total_download)
print("Total download: ", all_total_str)


#### Filter files

In [ ]:
#@markdown 

filterfiles()

#### Add single magnet

In [ ]:
import time
#@markdown

Magnet_Link = "" #@param {type:"string"}
Type = Types.Movie #@param ["Types.General", "Types.Movie", "Types.Show", "Types.Season", "Types.Episode"] {type:"raw"}
Sequential_Download = False #@param{type:"boolean"}

params = {"save_path": temp_save}

name = get_magnet_name(Magnet_Link)
handle = add_torrent_magnet(Magnet_Link, name, Type)

if Sequential_Download:
  handle.set_sequential_download(True)  


#### Choose seasons

In [ ]:
s_list=["S01","S02","S03","S04","S05","S06","S07","S08","S09","S10","S11","S12"]
check_list =["False","False","False","False","False","False","False","False","False","False","False","False"]
ok_list = [ ]
handle = active[-1]

S01=False #@param {type:"boolean"}
S02=False #@param {type:"boolean"}
S03=False #@param {type:"boolean"}
S04=False #@param {type:"boolean"}
S05=False #@param {type:"boolean"}
S06=False #@param {type:"boolean"}
S07=False #@param {type:"boolean"}
S08=False #@param {type:"boolean"}
S09=False #@param {type:"boolean"}
S10=False #@param {type:"boolean"}
S11=False #@param {type:"boolean"}
S12=False #@param {type:"boolean"}

check_list[0]=S01
check_list[1]=S02
check_list[2]=S03
check_list[3]=S04
check_list[4]=S05
check_list[5]=S06
check_list[6]=S07
check_list[7]=S08
check_list[8]=S09
check_list[9]=S10
check_list[10]=S11
check_list[11]=S12

for index, season in enumerate(s_list[:]):
  if check_list[index]:
    ok_list.append(season)

if(handle.has_metadata()):
  torinfo = handle.get_torrent_info()
  files = torinfo.files()

  for x in range(files.num_files()):
    if any(i in files.file_name(x) for i in ok_list):
      handle.file_priority(x,5)
    else:
      handle.file_priority(x,0)

  time.sleep(2)

  total_download = 0

  print("Download files:")
  print("")

  print("  "+handle.name())

  priorities = handle.file_priorities()
  torinfo = handle.get_torrent_info()
  files = torinfo.files()
  status= handle.status()
  file_sizes = status.total_wanted

  size_str = data_str(file_sizes)
  print("   Total download size: " + size_str)
  print("   Index Size    Name")

  fi_str=""
  for x in range(files.num_files()):
    if (priorities[x]>0):
      fs_str= data_str(files.file_size(x))
      fi_str = "   {:3d}-  ".format(x) + fs_str
      print(fi_str,files.file_name(x))

  print("")
  print("")

  print("  Total download: ", size_str)

else:
  print("    Metadata not recieved yet.")


#### Choose ranges

In [ ]:
#@markdown Choose files ranges

#@markdown 

handle = active[-1]

Start=0 #@param {type:"integer"}
End=0 #@param {type:"integer"}

if(handle.has_metadata()):
  torinfo = handle.get_torrent_info()
  files = torinfo.files()

  for index in range(len(priorities)):
    if index >= Start and index <= End:
      handle.file_priority(index,5)
    else:
      handle.file_priority(index,0)

  time.sleep(2)

  total_download = 0

  print("Download files:")
  print("")

  print("  "+handle.name())

  priorities = handle.file_priorities()
  torinfo = handle.get_torrent_info()
  files = torinfo.files()
  status= handle.status()
  file_sizes = status.total_wanted

  size_str = data_str(file_sizes)
  print("   Total download size: " + size_str)
  print("   Index Size    Name")

  fi_str=""
  for x in range(files.num_files()):
    if (priorities[x]>0):
      fs_str= data_str(files.file_size(x))
      fi_str = "   {:3d}-  ".format(x) + fs_str

      print(fi_str,files.file_name(x))

      
  print("")
  print("")

  print("  Total download: ", size_str)

else:
  print("   Metadata not recieved yet.")


In [ ]:
#@markdown Addtional Ranges

handle = active[-1]

Second_Range=False #@param {type:"boolean"}
Range1_Start=0 #@param {type:"integer"}
Range1_End=0 #@param {type:"integer"}

#@markdown 

#@markdown 

Third_Range=False #@param {type:"boolean"}
Range2_Start=0 #@param {type:"integer"}
Range2_End=0 #@param {type:"integer"}

#@markdown 

#@markdown 

Exlude_Range=False #@param {type:"boolean"}
Exlude_Range_Start=0 #@param {type:"integer"}
Exlude_Range_End=0 #@param {type:"integer"}


if(handle.has_metadata()):
  file_count = len(handle.file_priorities())
  torinfo = handle.get_torrent_info()
  files = torinfo.files()

  for index in range(file_count):
    if Second_Range and (index >= Range1_Start and index <= Range1_End):
      handle.file_priority(index,5)

    if Third_Range and (index >= Range2_Start and index <= Range2_End):
      handle.file_priority(index,5)

    if Exlude_Range and (index >= Exlude_Range_Start and index <= Exlude_Range_End):
      handle.file_priority(index,0)

  time.sleep(2)

  total_download = 0

  print("Download files:")
  print("")

  print("  "+handle.name())

  priorities = handle.file_priorities()
  torinfo = handle.get_torrent_info()
  files = torinfo.files()
  status= handle.status()
  
  file_sizes = status.total_wanted

  size_str = data_str(file_sizes)
  print("   Total download size: " + size_str)
  print("   Index Size    Name")

  fi_str=""
  for x in range(files.num_files()):
    if (priorities[x]>0):
      fs_str= data_str(files.file_size(x))
      fi_str = "   {:3d}-  ".format(x) + fs_str

      print(fi_str,files.file_name(x))

  print("")
  print("")

  print("  Total download: ", size_str)

else:
  print("   Metadata not recieved yet.")

#### Check choosen files

In [ ]:

#@markdown

#@markdown

total_download = 0

count_str = str(len(active))

print("Downloads: {0}".format(count_str))
for h in active:
  print("")
  print("  "+h.name())

  if(h.has_metadata()):
    priorities = h.file_priorities()
    torinfo = h.get_torrent_info()
    files = torinfo.files()
    status= h.status()
    file_sizes = status.total_wanted
    total_download+=file_sizes

    size_str = data_str(file_sizes)

    print("   Total download size: " + size_str)
    print("   Index Size    Name")

    fi_str=""
    for x in range(len(priorities)):
      if (priorities[x]>0):
        fs_str= data_str(files.file_size(x))
        fi_str = "   {:3d}-  ".format(x) + fs_str

        print(fi_str,files.file_name(x))
  else:
    print("    Metadata not recieved yet.")
    
print("")
all_total_str = data_str(total_download)
print("Total download: ", all_total_str)

#### Add trackers

In [ ]:
#@markdown 

#@markdown 

new_trackers = ["udp://tracker.opentrackr.org:1337/announce"
,"udp://tracker.internetwarriors.net:1337/announce"
,"udp://exodus.desync.com:6969/announce"
,"udp://tracker.cyberia.is:6969/announce"
,"udp://3rt.tace.ru:60889/announce"
,"http://5rt.tace.ru:60889/announce"
,"udp://explodie.org:6969/announce"
,"udp://47.ip-51-68-199.eu:6969/announce"
,"udp://opentracker.i2p.rocks:6969/announce"
,"http://open.acgnxtracker.com:80/announce"
,"udp://www.torrent.eu.org:451/announce"
,"udp://tracker.torrent.eu.org:451/announce"
,"udp://tracker.tiny-vps.com:6969/announce"
,"udp://tracker.ds.is:6969/announce"
,"udp://retracker.lanta-net.ru:2710/announce"
,"udp://open.stealth.si:80/announce"
,"udp://tracker.moeking.me:6969/announce"
,"udp://ipv4.tracker.harry.lu:80/announce"
,"http://tracker.dler.org:6969/announce"
,"http://rt.tace.ru:80/announce"
,"udp://93.158.213.92:1337/announce"
,"udp://138.255.103.83:1337/announce"
,"udp://208.83.20.20:6969/announce"
,"udp://193.218.118.220:6969/announce"
,"udp://79.111.118.59:60889/announce"
,"http://93.157.234.32:60889/announce"
,"udp://184.105.151.164:6969/announce"
,"udp://51.68.199.47:6969/announce"
,"udp://51.81.46.170:6969/announce"
,"udp://91.216.110.52:451/announce"
,"udp://89.234.156.205:451/announce"
,"udp://5.206.60.196:6969/announce"
,"udp://5.226.148.20:6969/announce"
,"udp://37.235.174.46:2710/announce"
,"udp://185.181.60.67:80/announce"
,"udp://138.201.150.56:6969/announce"
,"udp://51.15.40.114:80/announce"
,"http://168.119.237.9:6969/announce"
,"http://195.201.31.194:80/announce"
,"https://trakx.herokuapp.com:443/announce"
,"https://w.wwwww.wtf:443/announce"
,"https://tracker.tamersunion.org:443/announce"
,"https://tracker.sloppyta.co:443/announce"
,"https://tracker.renfei.net:443/announce"
,"https://tracker.nitrix.me:443/announce"
,"https://tracker.nanoha.org:443/announce"
,"https://tracker.lelux.fi:443/announce"
,"https://tracker.imgoingto.icu:443/announce"
,"https://tracker.hama3.net:443/announce"
,"https://tracker.foreverpirates.co:443/announce"
,"http://tracker2.wasabii.com.tw:6969/announce"
,"udp://tracker.sktorrent.net:6969/announce"
,"http://www.wareztorrent.com:80/announce"
,"udp://bt.xxx-tracker.com:2710/announce"
,"udp://tracker.eddie4.nl:6969/announce"
,"udp://tracker.grepler.com:6969/announce"
,"udp://tracker.mg64.net:2710/announce"
,"udp://wambo.club:1337/announce"
,"udp://tracker.dutchtracking.com:6969/announce"
,"udp://tc.animereactor.ru:8082/announce"
,"udp://tracker.justseed.it:1337/announce"
,"udp://tracker.leechers-paradise.org:6969/announce"
,"https://open.kickasstracker.com:443/announce"
,"udp://tracker.coppersurfer.tk:6969/announce"
,"http://87.253.152.137/announce"
,"http://91.216.110.47/announce"
,"http://91.217.91.21:3218/announce"
,"http://91.218.230.81:6969/announce"
,"http://93.92.64.5/announce"
,"http://atrack.pow7.com/announce"
,"http://bt.henbt.com:2710/announce"
,"http://bt.pusacg.org:8080/announce"
,"https://tracker.bt-hash.com:443/announce"
,"udp://tracker.leechers-paradise.org:6969"
,"https://182.176.139.129:6969/announce"
,"udp://zephir.monocul.us:6969/announce"
,"https://tracker.dutchtracking.com:80/announce"
,"https://grifon.info:80/announce"
,"udp://tracker.kicks-ass.net:80/announce"
,"udp://p4p.arenabg.com:1337/announce"
,"udp://tracker.aletorrenty.pl:2710/announce"
,"https://tracker.parrotsec.org:443/announce"
,"https://tracker.moxing.party:6969/announce"
,"https://tracker.ipv6tracker.ru:80/announce"
,"https://tracker.fastdownload.xyz:443/announce"
,"https://gwp2-v19.rinet.ru:80/announce"
,"https://tr.kxmp.cf:80/announce"
,"https://explodie.org:6969/announce"]


for h in active:
  trackers = h.trackers()
  #last_tier = trackers[-1]['tier']
  tier = 3
  for x in range(len(new_trackers)):
    annouce_entry = {'url': new_trackers[x], 'tier': tier}
    h.add_tracker(annouce_entry) 

#### List trackers

In [ ]:
#@markdown 

#@markdown 

handle = active[-1]
trackers = handle.trackers()
print("Trackers count ",len(trackers))

trackers = handle.trackers()
print("Trackers count ",len(trackers))

for x in trackers:
  #print(x)
  print(x['url'], x['tier'])

#### Start Download

In [ ]:
#@markdown

import time
from IPython.display import display
import ipywidgets as w

downloads= ses.get_torrents()
ses.resume()
active.clear() 
completed_views = []
active_views = []

progress_labels= []
progress_bars = []
states = []
download_speeds =[]
upload_speeds =[]
total_downloads =[]
total_uploads =[]

state_str = ["Queued","Checking","Downloading metadata","Downloading","Finished","Seeding","Allocating","Checking resume files"]

style0= {"padding-left":"200px","padding-right":"200px"}
layout_15 = w.Layout(width='auto', flex='15 1 0%', align_items="center")
layout_10 = w.Layout(width='auto', flex='10 1 0%', align_items="center")
layout_8 = w.Layout(width='auto', flex='8 1 0%', align_items="center")
layout_7 = w.Layout(width='auto', flex='7 1 0%', align_items="center")
layout_5 = w.Layout(width='auto', flex='5 1 0%', align_items="center")
layout_4 = w.Layout(width='auto', flex='4 1 0%', align_items="center")
layout_3 = w.Layout(width='auto', flex='3 1 0%', align_items="center")
layout_2 = w.Layout(width='auto', flex='2 1 0%', align_items="center")
layout_1 = w.Layout(width='auto', flex='1 1 0%', align_items="center")
layout_1E = w.Layout(width='auto', flex='1 1 0%', align_items="flex-start")

layout_up = {'flex_flow':'row','align_items':'center','width':'100%','justify_content':'space-around'}
layout_r = {'padding-left':'100px','flex_flow':'row','align_items':'flex-start','width':'95%','justify_content':'space-around'}
layout_c = {'width':'95%','align_items':'center','border':'solid #d9dbda 1px'}
layout_v = {'flex_flow':'row','align_items':'center','width':'95%','justify_content':'space-around'}

exit_btn = w.ToggleButton(
    disabled=False,
    button_style="danger",
    icon="check",
    value=False,
    layout=layout_1
)

empty_label_1= w.Label(value="",layout=layout_1)
empty_label_2= w.Label(value="",layout=layout_2)
empty_label_3= w.Label(value="",layout=layout_3)
empty_label_4= w.Label(value="",layout=layout_4)
empty_label_5= w.Label(value="",layout=layout_5)

total_torrents_label= w.Label(value="Total torrents:  {0}".format(len(downloads)),layout=layout_3)
active_label= w.Label(value="Active:  {0}".format(len(active)),layout=layout_3)
completed_label= w.Label(value="Completed:  {0}".format(len(completed)),layout=layout_3)

total_download_label= w.Label(value="Session download:  {0}".format(data_str(0)),layout=layout_5)
total_progress_label= w.Label(value="{0}%".format(0),layout=layout_3)
total_upload_label= w.Label(value="Session upload:  {0}".format(data_str(0)),layout=layout_5)

def Download_View(handle, type):
  status = handle.status()
  progress_bar = w.FloatProgress(
      value= status.progress*100,
      min=0,
      max=100,
      step=0.5,
      bar_style='info',
      orientation='horizontal',
      layout=layout_15)
  download_label = w.Label(value=handle.name(), layout = layout_10)
  progress_label= w.Label(value="{:.2f}%".format(status.progress*100), layout=layout_1)
  progress_bars.append(progress_bar)
  progress_labels.append(progress_label)
  strs=[
        "Download Speed:    {0}".format(speed_str(status.download_rate)),
        "Upload speed:    {0}".format(speed_str(status.upload_rate)),
        "Total Download:    {0} / {1}".format(data_str(status.total_wanted_done),data_str(status.total_wanted)),
        "Total Upload:    {0}".format(data_str(status.total_upload))]  #All time upload dowsn't work properly

  ls = [w.Label(value = v, layout = layout_4) for v in strs]
  ls.insert(0, w.Label(value = state_str[status.state], layout=layout_2))
  ls.insert(1, w.Label(value = type.name, layout=layout_2))
  ls.insert(6,empty_label_2)

  states.append(ls[0])
  download_speeds.append(ls[2])
  upload_speeds.append(ls[3])
  total_downloads.append(ls[4])
  total_uploads.append(ls[5])

  r1 = w.Box([ download_label,progress_label, progress_bar,empty_label_1],layout=layout_r,style=style0)
  r2 = w.Box(ls,layout=layout_r,style=style0)
  c = w.VBox([r1,r2],layout=layout_c)
  v = w.Box([c, empty_label_1],layout=layout_r,style=style0)
  return c

def Complete_View(handle, type):
  status = handle.status()
  progress_bar = w.FloatProgress(value= 100, min=0, max=100, step=50, bar_style='success', orientation='horizontal', layout=layout_15 )
  download_label = w.Label(value = handle.name(), layout = layout_10)
  progress_label= w.Label(value = "100%",layout = layout_1E)
  upload = status.all_time_upload
  if upload == 0:
    upload = status.total_upload
  strs=[
        "Completed at:    {0}".format(time.strftime('%I:%M:%S %p', time.localtime(status.completed_time))),
        "Total time:     {:.2f} m".format(status.active_time/60),
        "Download:    {0} / {1}".format(data_str(status.total_wanted_done),data_str(status.total_wanted)),
        "Upload:    {0}".format(data_str(upload)) ]

  ls = [w.Label(value = v, layout = layout_4) for v in strs]
  ls.insert(0, w.Label(value = "Completed", layout=layout_2))
  ls.insert(1, w.Label(value = type.name, layout=layout_2))
  ls.insert(6,empty_label_3)

  r1 = w.Box([ download_label,progress_label, progress_bar,empty_label_1],layout=layout_r,style=style0)
  r2 = w.Box(ls,layout=layout_r,style=style0)
  c = w.VBox([r1,r2],layout=layout_c)
  return c

def Update_View(i, status):
  states[i].value = state_str[status.state]
  progress_labels[i].value =  "{:.2f}%".format(status.progress*100)
  progress_bars[i].value = status.progress*100
  download_speeds[i].value= "Download Speed:    {0}".format(speed_str(status.download_rate))
  upload_speeds[i].value= "Upload speed:    {0}".format(speed_str(status.upload_rate))
  total_downloads[i].value= "Download:    {0} / {1}".format(data_str(status.total_wanted_done),data_str(status.total_wanted))
  total_uploads[i].value= "Upload:    {0}".format(data_str(status.total_upload))
  return

total_download = 0
total_wanted = 0
total_up = 0
completed_down = 0
completed_up = 0
total_progress = 0

def Update_Stats(_download,_upload):
  if total_wanted == 0:
    total_progress = 0
  else:
    total_progress = (100*_download)/total_wanted 
  active_label.value = "Active:  {0}".format(len(active))
  completed_label.value = "Completed:  {0}".format(len(completed))
  total_download_label.value = f"Session download:   {data_str(_download)} / {data_str(total_wanted)}"
  total_progress_label.value = f"     {round(total_progress,2)}%"
  total_upload_label.value = "Session upload:   {}".format(data_str(_upload))

def Remove_Active_View(i):
  active_views[i].layout.display = 'none'
  del active_views[i]
  del progress_labels[i]
  del progress_bars[i]
  del download_speeds[i]
  del upload_speeds[i]
  del total_downloads[i]
  del total_uploads[i]
  return

def Completed_Stats():
  _download=0
  _upload=0
  for c in completed:
    status = c.status()
    upload = status.all_time_upload
    if upload == 0:
      upload = status.total_upload  
    _download+= status.total_wanted_done
    _upload+= upload
  return [_download,_upload]

for handle in downloads:
  status = handle.status()
  total_wanted+=status.total_wanted
  key =  str(handle.info_hash())
  type = downloads_dict[key]["type"]
  if handle in completed:
    completed_views.append(Complete_View(handle, type))
    continue
  if handle not in active:
    active.append(handle)
  active_views.append(Download_View(handle, type))
  handle.resume()
    
active_down = 0 
active_up = 0 
stat= Completed_Stats()
completed_down=stat[0]
completed_up=stat[1]
Update_Stats(completed_down, completed_up)
stats_row = w.Box([total_torrents_label, active_label, completed_label, total_download_label, total_progress_label, total_upload_label, empty_label_3],layout=layout_up,style=style0)
exit_row = w.Box([ empty_label_1, exit_btn],layout=layout_r,style=style0)
session_state = "SESSION IS RUNNING               General save: " + drive_path + "           Temp save: " + temp_save
print("\n\n\n                       " + session_state)

display(stats_row)
print("\n")
display(*active_views)
print("\n")
display(*completed_views)
while active:
  if exit_btn.value:
    print(exit_btn.value)
    break
  shift = 0
  active_down = 0
  active_up = 0
  active_wanted = 0
  for index, handle in enumerate(active[:]):    
    status = handle.status()
    if handle.is_finished():
      key =  str(handle.info_hash())
      type = downloads_dict[key]["type"]
      handle.pause()
      name = handle.name()
      new_name = ""
      p1 = status.save_path + "/" + name
      p0 = status.save_path
      if not os.path.isdir(p1):
        p0 = status.save_path + "/" + os.path.splitext(name)[0]
        os.makedirs(p0, exist_ok= True)
        !mv "$p1" "$p0"
        p1 = p0
      shave (handle, p0, p1)
      if type == Types.Movie:
          new_name = name_movie(name)
          save_path = Movies_Path
          after_shave(p1, new_name)
      elif type == Types.Show:
          new_name = name_show(name)
          clean_tv(p1)
          save_path = Shows_Path
          check = save_path + "/" + new_name
      elif type == Types.Season:
          new_name = name_season(name)
          show = name_show(name)
          clean_season(p1)
          save_path= Shows_Path + "/" + show
          os.makedirs(save_path, exist_ok= True)
      elif type == Types.Episode:
          new_name = name_episode(name)
          show = name_show(name)
          season= name_season(name)
          save_path= Shows_Path + "/" + show + "/" + season
          os.makedirs(save_path, exist_ok= True)
      else:
          save_path = drive_path

      p2 = done_path + "/" + new_name
      !mv "$p1" "$p2"

      if not new_name:
        p2 = p2 + name

      if type == Types.Episode and os.path.isdir(p2):
        clean_season(p2)
        p2 = p2 + "/" 
        
      if type == Types.Show and os.path.isdir(check):
        save_path = check
        p2 = p2 + "/"

      !mv "$p2"* "$save_path"

      Remove_Active_View(index-shift)
      completed.append(handle)
      active.remove(handle)
      complete_view = Complete_View(handle, type)
      display(complete_view)
      completd_stat= Completed_Stats()
      completed_down=completd_stat[0]
      completed_up=completd_stat[1]
      shift= shift + 1
    else:
      Update_View(index-shift, status)
      active_up+=status.total_upload
      active_down+=status.total_wanted_done
      active_wanted+=status.total_wanted

  total_wanted = completed_down + active_wanted
  total_download = completed_down + active_down
  total_up = active_up + completed_up
  Update_Stats(total_download, total_up)
  time.sleep(0.9)

#### Pause session

In [ ]:
#@markdown 

#@markdown 

downloads= ses.get_torrents()
for h in downloads:
  try:
    print(h.name())
    h.pause()
  except:
    print("Invailed torrent handle.")
    
ses.pause()

#### Save download progress

In [ ]:
#@markdown 

Save_To = "/content/drive/Shareddrives/SM1/Downloads/.uncomplete" #@param {type: "string"}

def sv():
  if active:
    print(" Choose a download to save progress , q to exit:\n")
    for i in range(len(active)):
      print("   {0}-  {1}".format(i+1, active[i].name()))
    print()
    inpt =  input("")
    if inpt == 'q':
      return
    choice = int(inpt) - 1
    if "," in choice:
      splitted = choice .split(",")
      choices = splitted
    else:
      choices = [choice]

    for choice in choices:
      if(choice >= len(active) or choice < 0):
        print("\nThe choice is out of range.\n")
      else:
        tr_name = active[choice].name()
        print("\nSaving {0} into {1}".format(tr_name, Save_To))
        print()
        path = temp_save +"/"+ tr_name
        !cp -r "$path" "$Save_To"
        print("..................")
        print("       Done")

sv()

#### Verify saving to drive

In [ ]:
#@markdown

from genericpath import sameopenfile
from tkinter.font import names
from xml.dom.expatbuilder import Namespaces


downloads= ses.get_torrents()
not_copied=[]
for download in downloads:
    name = download.name()
    print("  Checking: {0}".format(name) )
    status = download.status()
    byte_check = status.total_wanted - status.total_wanted_done
    if download.is_finished() or byte_check == 0:
        copy_status = ""
        if movie_exists(name):
            copy_status ="copied"
        else:
            copy_status = "not copied"
            not_copied.append(download)
        print("  Status:   completed   {0}".format(copy_status))
    else:
        print("  Status:   not completed   state: {0}     progress: {1}%     downloaded: {2}     remained: {3}".format(status.state, round(status.progress*100,2), data_str(status.total_wanted_done), data_str(status.total_wanted-status.total_wanted_done)))
    print()
print(  "Downloads completed but not copied:")
if not not_copied:
    print("     None")
else:
    for index in range(len(not_copied)):
        print("     {0}- {1}".format( index + 1, not_copied[index].name()))
print()
if not_copied:
  print("Do you want to copy and complete saving these downloads? y/n")
  print()
  choice = input("")
  if choice == 'y':
    for handle in not_copied:
      new_name = ""
      if type == Types.Movie:
        new_name = name_movie(name)
        save_path = Movies_Path
      elif type == Types.Show:
        new_name = name_show(name)
        save_path = Shows_Path
      elif type == Types.Season:
        new_name = name_season(name)
        show = name_show(name)
        save_path= Shows_Path + "/" + show
        os.makedirs(save_path, exist_ok= True)
      elif type == Types.Episode:
        new_name = name_episode(name)
        show = name_show(name)
        season= name_season(name)
        save_path= Shows_Path + "/" + show + "/" + season
        os.makedirs(save_path, exist_ok= True)
      else:
        save_path = drive_path
      status = handle.status()
      p1 = status.save_path + "/" +  handle.name()
      p2 = done_path + "/" + new_name
      !mv "$p1" "$p2"
      if not new_name:
        p2 = p2 + handle.name()
      !mv "$p2" "$save_path"
      print(".................")
      print("       Done")


#### Clear Completed

In [ ]:
#@markdown

for download in completed:
  print("Deleting: " + download.name())
  ses.remove_torrent(download, 0)

completed.clear()


#### Delete all completed 

In [ ]:
#@markdown 

#@markdown

for f in completed:
  try:
    ses.remove_torrent(f)
  except:
    print("Invild torrent.")
    continue
completed.clear()

#### Clear active

In [ ]:
#@markdown 

for f in active:
  f.pause()
  active.remove(f)
  try:
    ses.remove_torrent(f)
  except:
    print("Invalid torrent.")
  
active.clear()

#### Copy and move files

In [ ]:
#@markdown 

From_Path = "/content/drive/MyDrive/Library/Movies" #@param {type: "string"}
To_Path = "/content/drive/Shareddrives/SM1/Library/Movies" #@param {type: "string"}

Move = False #@param {type:"boolean"}
Rename_only = False #@param {type:"boolean"}
Movie_Name = True #@param {type:"boolean"}
Show_Name = False #@param {type:"boolean"}

def cp_mv(content):
  if content:
    print()
    print(" Enter directory range to copy [Format: start-end  All: . Exit: q]:")
    print_files(content, 0 ,len(content))
    print()
    start = -1
    end = -1
    rng = input("")
    print()
    if rng.lower() == 'q':
      return
    elif rng == '.':
      start = 0
      end = len(content)
    elif not '-' in rng:
      end = int(rng)
      start = end - 1
    else:
      rngs = rng.split('-')
      start = int(rngs[0])-1
      end = int((rngs[1]))
    print("Chosen range start: {0}, end: {1}".format(start+1,end))
    print()
    if end > len(content) or start < 0 or ((start + end) < 0) :
      print("The ranges' ends are out of the directory file range.")
      return
    elif Rename_only:
      print("Renaming these files")
      print()
      print_files(content,start,end)
      print()
      for i in range(start, end):
        name  = ""
        if Movie_Name:
          name = name_movie(content[i])
        elif Show_Name:
          name = name_show(content[i])
        else:
          print("  Enter a new name:")
          name = input("")
        if not name:
          continue
        print("   Renaming   {0}   new name:   {1}".format(content[i], name))
        p1 = From_Path + "/" + content[i]
        p2 = From_Path + "/" + name 
        !mv "$p1" "$p2"

    elif Move:
      tp= From_Path + "/.moved"
      os.makedirs(tp, exist_ok = True)
      print("Moving these files to {0}".format(To_Path))
      print()
      print_files(content,start,end)
      print()
      for i in range(start, end):
        rename  = ""
        if Movie_Name:
          rename = "/" + name_movie(content[i])
        elif Show_Name:
          rename =  "/" + name_show(content[i])

        print("   Moving  {0}".format(content[i]))

        p1 = From_Path + "/" + content[i]
        p2 = tp +  rename

        !mv "$p1" "$p2"
        if not rename:
          p2 = p2 + content[i]
        !cp -r "$p2" "$To_Path"
    
    else:
      print("Copying these files to {0}".format(To_Path))
      print()
      print_files(content,start,end)
      print()
      for i in range(start, end):
        rename  = ""
        if Movie_Name:
          rename = "/" + name_movie(content[i])
        elif Show_Name:
          rename =  "/" + name_show(content[i])    

        p0 = To_Path + content[0]
        p1 = From_Path + "/" + content[i]
        p2 = To_Path + rename

        if os.path.exists(p0) or (rename and os.path.exists(p2)) :
          print("   Skipping:  {0}  directory alraedy exists.".format(content[i]))
          continue

        print("   Copying:  {0}".format(content[i]))
        !cp -r "$p1" "$p2"

    print()
    print("   .....................")
    print("           Done")
    return
  else:
    print("Empty directory.")
    return

drfiles = filter_dirs(next(os.walk(From_Path))[1])
drfiles.sort()
cp_mv(drfiles)

#### Rm

In [ ]:
#@markdown 
Delation_Path = "/content/drive/Sharedsrives/SM1/Downloads/d-01" #@param {type: "string"}

def rm(content):
  if content:
    print()
    print(" Enter directory range to copy (start-end):")
    print_files(content, 0 ,len(content))
    print()
    start = -1
    end = -1
    rng = input("")
    print()
    if rng == 'q':
      return
    if not '-' in rng:
      start = int(rng)
    else:
      rngs = rng.split('-')
      start = int(rngs[0])-1
      end = int((rngs[1]))
    print("Chosen range start: {0}, end: {1}".format(start+1,end))
    print()
    if end >= len(content) or start < 0 or ((start + end) < 0) :
      print("The ranges' ends are out of the directory file range.")
      return
    else:
      print("Deleting these directories".format(To_Path))
      print()
      print_files(content,start,end)
      print()
      for i in range(start, end):
        print("   Deleting  {0}".format(content[i]))
        path = Delation_Path+"/"+content[i]
        !rm -r "$path"

    print()
    print("   ......................")
    print("           Done")
    return
  else:
    print("Empty directory.")
    return

drfiles = filter_dirs(next(os.walk(Delation_Path))[1])
drfiles.sort()
rm(drfiles)

#### Clear temp

In [ ]:
#@markdown
#  
Delete_torrent_files = True #@param {type:"boolean"}
Clear_done_folder = True #@param {type:"boolean"}
Clear_temp_folder = False #@param {type:"boolean"}

if Delete_torrent_files:
  !find . -maxdepth 1 -name "*.torrent" -type f -delete 

if Clear_done_folder:  
  !rm -r "/content/done/"*

if Clear_temp_folder:
  !rm -r "$temp_save/"*  

#### Check general status

In [ ]:
#@markdown 

#@markdown 

from IPython.display import display
import ipywidgets as widgets

downloads= ses.get_torrents()

session_state = ""
if ses.is_paused() :
  session_state = "SESSION IS PAUSED"
else:
  session_state = "SESSION IS RUNNING"

print("            " + session_state)

layout = {'width':'auto', 'margin-left':'50px'}
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

for index, download in enumerate(downloads[:]):
    bar = download_bars[index]
    shift = 0
    try:
      status = download.status()
      state = ""
      if status.paused == True:
        state = "PAUSED"
      else:
        state_str[status.state]
      bar.description = " ".join(
          [
              download.name(),
              speed_str(ByteToMB(status.download_rate)),
              state,
              "{0} / {1}".format(status.total_wanted_done, status.total_wanted)
          ]
      )
      bar.value = round(status.progress * 100,2)
    except:
      downloads.remove(download)
      download_bars.remove(index-shift)
      shift = shift + 1
      print("Invalid torrent")
      continue


#### Resume all

In [ ]:
#@markdown 

ses.resume()

for h in active:
  h.resume()

#### Remove torrent

In [ ]:
#@markdown

downloads= ses.get_torrents()

print("Please select a torrent to remove:")
print("")

for index,h in enumerate(downloads[:]):
  print("{0}- {1}".format(index + 1, h.name()))

print("Enter torrent selection:  ")
select = input("")

if "," in select:
  selects = select.split(",")
else:
  selects = [select]

for select in selects:
  select = int(select) - 1
  h = downloads[select]
  print("\nDeleting {0}".format(h.name()))
  print("...........\n")
  h.pause()
  if h in active:
    active.remove(h)
  if h in completed:
    active.remove(h)
  try:
    ses.remove_torrent(h)
    del downloads_dict[str(h.info_hash())]
  except:
    print("Selected torrent is invalid.")  
    
print("Done.")

#### Clear downloads

In [ ]:
#@markdown 

#@markdown 

downloads= ses.get_torrents()

for download in downloads:
    print("Deleting: " + download.name())
    ses.remove_torrent(download, 0)

active.clear()
completed.clear()
downloads_dict.clear()

#### Clean movies

In [ ]:
#@markdown

Path = "/content/drive/Shareddrives/SM1/Library/Movies/Finding.Forrester.2000.1080p.BluRay.x265-RARBG" #@param {type: "string"}

if Path:
  name = Path.split("/")[-1]
  clean_movie(Path, name)
else:   
  clean_movies_dir()

#### Clean Shows

In [ ]:
#@markdown

Path = "" #@param {type: "string"}

if Path:
    clean_tv(Path)
else:
    clean_shows()